In [1]:
import csv
import numpy as np
import datetime
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize
from dateutil import rrule 

DAY = 86400000000000

# Generate ruleset for holiday observances on the NYSE

def NYSE_holidays(a=datetime.date.today(), b=datetime.date.today()+datetime.timedelta(days=365)):
    rs = rrule.rruleset()

    # Include all potential holiday observances
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=31, byweekday=rrule.FR)) # New Years Day  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 1, bymonthday= 1))                     # New Years Day  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 1, bymonthday= 2, byweekday=rrule.MO)) # New Years Day    
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 1, byweekday= rrule.MO(3)))            # Martin Luther King Day   
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 2, byweekday= rrule.MO(3)))            # Washington's Birthday
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, byeaster= -2))                                  # Good Friday
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 5, byweekday= rrule.MO(-1)))           # Memorial Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 7, bymonthday= 3, byweekday=rrule.FR)) # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 7, bymonthday= 4))                     # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 7, bymonthday= 5, byweekday=rrule.MO)) # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 9, byweekday= rrule.MO(1)))            # Labor Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=11, byweekday= rrule.TH(4)))            # Thanksgiving Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=24, byweekday=rrule.FR)) # Christmas  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=25))                     # Christmas  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=26, byweekday=rrule.MO)) # Christmas 
    
    # Exclude potential holidays that fall on weekends
    rs.exrule(rrule.rrule(rrule.WEEKLY, dtstart=a, until=b, byweekday=(rrule.SA,rrule.SU)))

    return rs
    
# Generate ruleset for NYSE trading days

def NYSE_tradingdays(a=datetime.date.today(), b=datetime.date.today()+datetime.timedelta(days=365)):
    rs = rrule.rruleset()
    rs.rrule(rrule.rrule(rrule.DAILY, dtstart=a, until=b))
    
    # Exclude weekends and holidays
    rs.exrule(rrule.rrule(rrule.WEEKLY, dtstart=a, byweekday=(rrule.SA,rrule.SU)))
    rs.exrule(NYSE_holidays(a,b))
    
    return rs


# Count NYSE holidays days in next 4 years
hdays =list(NYSE_holidays(datetime.datetime(2011,1,1),datetime.datetime(2014,12,31)))

fields = ['caldt','spindx']
sp500 = pd.read_csv('./Datasets/S&P500.csv', skipinitialspace=True, usecols=fields)
sp500['caldt'] = pd.to_datetime(sp500['caldt'], format='%m/%d/%Y', utc=True)
sp500.rename(index=str, columns={"caldt": "date"}, inplace=True)

date_col = 'date'
sym_col = 'symbol'
exp_date_col = 'exdate'
strike_price_col = 'strike_price'
best_bid_col = 'best_bid'
best_ask_col = 'best_offer'
imp_vol_col = 'impl_volatility'
volumn_col = 'volume'
tenor_col = 'Tenor'
indx_col = 'spindx'

fields = [date_col, sym_col, exp_date_col, strike_price_col, best_bid_col, best_ask_col, imp_vol_col, volumn_col]

#Read options_data
df = pd.read_csv('./Datasets/option_data.csv', skipinitialspace=True, usecols=fields)

#Extract spxw
spxw = df[df[sym_col].str.contains('SPXW')]

#Format Dates
df[date_col] = pd.to_datetime(df[date_col], format='%m/%d/%Y', utc=True)
df[exp_date_col] = pd.to_datetime(df[exp_date_col], format='%m/%d/%Y', utc=True)

#Create Tenor
df[tenor_col]=df[exp_date_col]-df[date_col]
df = pd.merge(df, sp500, on=date_col, how='left')

In [2]:
#Build Moneyness
sLength = len(df[date_col])
abs_diff_col = 'abs_diff'
moneyness_col = 'moneyness'
IVATM_col = 'IVATM'

df[abs_diff_col] = abs(df[strike_price_col] - 1000 * df[indx_col])
#get the IVATM for each option grouped by start date and tenor (expiry date)
IVATM = df.loc[df.groupby([date_col, exp_date_col])[abs_diff_col].idxmin(), [date_col, exp_date_col, imp_vol_col]]
IVATM.rename(index=str, columns={imp_vol_col: IVATM_col}, inplace=True)
df = pd.merge(df, IVATM, on=[date_col, exp_date_col],  how='left')
df[moneyness_col] = np.log(df[strike_price_col] / (1000 * df[indx_col])) / \
    (np.sqrt((df[tenor_col].values / DAY).astype(float) / 365) * df[IVATM_col])

In [3]:
#Delete 0 volume entries
df=df[df[volumn_col]>0]

#Tenor criterion
# at least 10 different strikes across the same tenor
counts = df.groupby([date_col, exp_date_col]).size().reset_index(name='counts')
valid_tenors = counts[counts['counts'] > 9][[date_col, exp_date_col]]

#front maturity contract maturity must be less than 9 days and have moneyness less than -3.5
front_maturities = df.loc[df.groupby([date_col])[tenor_col].idxmin(), [date_col, tenor_col]]
front_maturities = front_maturities[(front_maturities[tenor_col].values / DAY).astype(int) < 9]
front_contracts = pd.merge(df, front_maturities, on=[date_col, tenor_col], how="inner")
front_moneyness = front_contracts[[date_col, tenor_col, moneyness_col]] \
    .groupby([date_col, tenor_col]).agg(np.min).reset_index()
valid_dates = front_moneyness[front_moneyness[moneyness_col] < -3.5][[date_col]]

df = pd.merge(df, valid_tenors, on=[date_col, exp_date_col], how="inner")
df = pd.merge(df, valid_dates, on=[date_col], how="inner")

#Maturity criterion
df=df[(df[tenor_col].values / DAY).astype(int) < 365]

#Moneyness criterion
df=df[df[moneyness_col]>-15]
df=df[df[moneyness_col]<5]

#Bid criterion
df=df[df[best_ask_col]/(0.001 + df[best_bid_col])<5]

#Holiday criterion: this is probably already taken care of by the data
# for h in hdays:
#     df=df.loc[(df[date_col]!=h.tz_localize(None))]

In [4]:
#Separate short dated
short_dated=df[(df[tenor_col].values / DAY).astype(int) < 10]
long_dated=df[(df[tenor_col].values / DAY).astype(int) >= 10]

#for short dated contract, only retain the shortest maturity
front_short = short_dated.loc[short_dated.groupby([date_col])[tenor_col].idxmin(), [date_col, tenor_col]]
short_dated = pd.merge(short_dated, front_short, on=[date_col, tenor_col], how="inner")

#money ness is between -8 and 5
short_dated = short_dated[(short_dated[moneyness_col] > -8) & (short_dated[moneyness_col] < 5)]

#at least 5 traded each day
short_dated = short_dated[short_dated[volumn_col] > 4]

In [337]:
################################################################################# 
## getting the transform of the price
#################################################################################
''' 
with the 2 factor stochastic volatility jump model the solution has the form 
 exp(alpha + beta_0 * X + beta_1 * V1 + beta_2 * V2), where beta_0 = u, and beta_1, beta_2 is the same for all
 compensators for the SVJ models v(dt, dx, dy)
 alpha = alpha0 + terms depending on compensator
'''

def beta(tau, u, kappa, sigma, rho):
    a = -u**2
    b = sigma * rho * u - kappa
    gamma = np.sqrt(b**2 + a * sigma**2)
    return -a * (1 - np.exp(-gamma * tau)) / (2 * gamma - (gamma + b) * (1 - np.exp(-gamma * tau)))

def alpha0(tau, u, kappa1, sigma1, rho1, vbar1, kappa2, sigma2, rho2, vbar2, r, div_rate):
    a = -u**2
    b1 = sigma1 * rho1 * u - kappa1
    gamma1 = np.sqrt(b1**2 + a * sigma1**2)
    b2 = sigma2 * rho2 * u - kappa2
    gamma2 = np.sqrt(b2**2 + a * sigma2**2)
    
    return -r * tau + (r - div_rate) * u * tau \
        - kappa1 * vbar1 * ((gamma1 + b1) / (sigma1**2) * tau + \
                2 / (sigma1**2) * np.log(1 - (gamma1 + b1) * (1 - np.exp(- gamma1 * tau)) / (2 * gamma1))) \
        - kappa2 * vbar2 * ((gamma2 + b2) / (sigma2**2) * tau + \
                2 / (sigma2**2) * np.log(1 - (gamma2 + b2) * (1 - np.exp(- gamma2 * tau)) / (2 * gamma2)))    

# the SVJJ model by Duffie et. al. (2000)
def int_theta_SVJJ(tau, u, mu_x, sigma_x, mu_y, rho_j, kappa, sigma, rho):
    a = -u**2
    b = sigma * rho * u - kappa
    gamma = np.sqrt(b**2 + a * sigma**2)
    c = 1 - rho_j * mu_y * u
    d = (gamma - b) / ((gamma - b) * c + mu_y * a) * tau \
        - 2 * mu_y * a / ((gamma * c)**2 - (b * c - mu_y * a)**2) * \
            np.log(1 - ((gamma + b) * c - mu_y * a) / (2 * gamma * c) * (1 - np.exp(-gamma * tau)))
    
    return np.exp(mu_x * u + 0.5 * sigma**2 * u**2) * d

# the transform of state (X, V1, V2) with SVJJ model
def phi_SVJJ(u, X, V1, V2, t, T, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
             eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j):
    tau = T - t
    lam = eta0 + eta1 * V1 + eta2 * V2
    a0 = alpha0(tau, u, kappa1, sigma1, rho1, vbar1, kappa2, sigma2, rho2, vbar2, r, div_rate)
    int_theta = int_theta_SVJJ(tau, u, mu_x, sigma_x, mu_y, rho_j, kappa1, sigma1, rho1)
    a = a0 - lam * tau + lam * int_theta
    b0 = u
    b1 = beta(tau, u, kappa1, sigma1, rho1)
    b2 = beta(tau, u, kappa2, sigma2, rho2)
    
    return np.exp(a + b0 * X + b1 * V1 + b2 * V2)

##########################################################################################
## Calculate the price using the inverse transform 
##########################################################################################

def calc_G_SVJJ(a, b, y, X, V1, V2, T, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                 eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j):
    # y can be an array of log strike prices
    t = 0
    upperbound = 500
    sep = 1
    sep2 = 10
    sep3 = 100
    step_size1 = 0.001
    step_size2 = 0.01
    step_size3 = 0.05
    step_size4 = 0.5

    step_sizes = np.concatenate((step_size1 * np.ones(int(sep / step_size1)), \
                                 step_size2 * np.ones(int((sep2 - sep) / step_size2)), \
                                 step_size3 * np.ones(int((sep3 - sep2) / step_size3)),
                                step_size4 * np.ones(int((upperbound - sep3) / step_size4))))
    v_arr1 = np.arange(start=0, stop=sep, step=step_size1)
    v_arr2 = np.arange(start=sep, stop=sep2, step=step_size2)
    v_arr3 = np.arange(start=sep2, stop=sep3, step=step_size3)
    v_arr4 = np.arange(start=sep3, stop=upperbound+step_size4, step=step_size4)
    v_arr = np.concatenate((v_arr1, v_arr2, v_arr3, v_arr4))
    u_arr = a + b * v_arr * 1j
    phi_arr = phi_SVJJ(u_arr, X, V1, V2, t, T, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                 eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j)
    inv_arr = np.exp(-np.outer(y, v_arr) * 1j)
    temp = np.imag(phi_arr * inv_arr) / (v_arr + 1e-15)
    #trapezoidal rule
    integr = np.sum((temp[:,0:-1] + temp[:,1:]) / 2 * (step_sizes), axis=1) / np.pi
    return phi_SVJJ(a, X, V1, V2, t, T, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                 eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j) / 2 - integr

def price_SVJJ(y, X, V1, V2, T, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                 eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j):
    G0 = calc_G_SVJJ(1e-7, 1, y, X, V1, V2, T, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                 eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j)
    G1 = calc_G_SVJJ(1, 1, y, X, V1, V2, T, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                 eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j)
    return np.exp(y) * G0 - G1


#Black Scholes volatility
def BSIV(X, y, r, T, price):
    # Find BSIV using Newton's method
    sigma = 0.3
    sigma_old = sigma
    for i in range(10):
        d1 = (X - y + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        BS_price = norm.cdf(-d2) * np.exp(y - r * T) - norm.cdf(-d1) * np.exp(X)
        vega = np.exp(y - r * T) * norm.pdf(d2) * np.sqrt(T)
        sigma = sigma_old - (BS_price - price) / vega
        if (abs(sigma - sigma_old) < 1e-6):
            break
        if (sigma > 2 | pd.isnull(sigma)): #bound the volatility to prevent code from breaking in edge cases
            sigma = 2
        if (sigma <= 0):
            sigma = 0.01
        sigma_old = sigma
    return sigma

# vectorized BSIV, with y and p as vector and returns a vector
def BSIV_vec(X, y, r, T, price):
    sigma = 0.3 * np.ones(price.shape[0])
    sigma_old = sigma
    for i in range(10):
        d1 = (X - y + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        BS_price = norm.cdf(-d2) * np.exp(y - r * T) - norm.cdf(-d1) * np.exp(X)
        vega = np.exp(y - r * T) * norm.pdf(d2) * np.sqrt(T)
        sigma = sigma_old - (BS_price - price) / vega
        if (np.max(np.absolute(sigma - sigma_old)) < 1e-6):
            break
        sigma[pd.isnull(sigma)] = 2
        sigma[sigma > 2] = 2
        sigma[sigma <= 0] = 0.01
        sigma_old = sigma
    return sigma

In [79]:
#Using bi-power estimator for non-parametric diffusion estimate
fields = ['caldt','spindx']
sp500_m = pd.read_csv('./Datasets/sp500_more.csv', skipinitialspace=True, usecols=fields)
sp500_m['caldt'] = pd.to_datetime(sp500_m['caldt'], format='%m/%d/%Y', utc=True)
sp500_m.rename(index=str, columns={"caldt": "date"}, inplace=True)

sp500_m['log_ind'] = sp500_m[indx_col].apply(np.log)
sp500_m['log_diff'] = abs(sp500_m[['log_ind']].diff())
sp500_m['prod_log_diff'] = sp500_m['log_diff'] * sp500_m['log_diff']
sp500_m['V_est'] = sp500_m['prod_log_diff'].rolling(30).mean() * 252
sp500_m = sp500_m[sp500_m[date_col] > '2011-01-01']

In [406]:
# Cut down number of samples to estimate parameters
estimation_dates = long_dated[[date_col]].drop_duplicates()
estimation_dates = estimation_dates[estimation_dates[date_col] < '2015-01-01']
def is_wed(dt):
    return np.is_busday(dt, weekmask='0010000')
estimation_dates = estimation_dates[estimation_dates[date_col].apply(is_wed)].reset_index()

est_dat = pd.merge(long_dated, estimation_dates, on=date_col, how="inner")
# fill in missing BSIV in dataset
for i in range(est_dat.shape[0]):
    if pd.isnull(est_dat.iloc[i][imp_vol_col]):
        mid_p = (est_dat.iloc[i][best_bid_col] + est_dat.iloc[i][best_ask_col]) / 2
        est_dat.at[i, imp_vol_col] = BSIV(np.log(est_dat.at[i, indx_col]), \
                                            np.log(est_dat.at[i, strike_price_col] / 1000), \
                                            r, T, mid_p)

In [421]:
###########################################################
## Calculating the objective functions
###########################################################
def MSE(option_groups, X, V1, V2, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                     eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j, Nt, V_est):
    sum_sq = 0
    sum_err = 0
    for exp_date, same_tenor_options in option_groups:
        K_arr = same_tenor_options[strike_price_col].values / 1000
        y_arr = np.log(K_arr)
        T = same_tenor_options[tenor_col].tolist()[0].days / 365

        BSIV_arr = np.zeros(K_arr.shape[0])
        p = price_SVJJ(y_arr, X, V1, V2, T, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                     eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j)
        BSIV_arr = BSIV_vec(X, y_arr, r, T, p)

        sum_sq += np.sum(np.square(BSIV_arr - same_tenor_options[imp_vol_col].values))
        sum_err += np.sum(BSIV_arr - same_tenor_options[imp_vol_col].values)

    err1 = sum_sq / Nt
    err2 = 2 * 30 / Nt * (np.sqrt(V_est) - np.sqrt(V1 + V2))**2 / V_est
    
    return {'avg_err': err1 + err2, 'err_sum_sq': sum_sq, 'err_sum': sum_err, 'avg_err_sum_sq' : err1}
    
def get_volatility_states(option_dat, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                 eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j):
    g = option_dat.groupby([date_col])
    mse = 0
    num_dates = option_dat[[date_col]].drop_duplicates().shape[0]
    V1_arr = np.zeros(num_dates)
    V2_arr = np.zeros(num_dates)
    ind = 0
    for date, options in g:
        g2 = options.groupby([exp_date_col])
        Nt = options.shape[0]
        X = np.log(options[indx_col].tolist()[0])
        
        if (ind == 0):
            V1 = vbar1
            V2 = vbar2
        else:
            V1 = V1_arr[ind-1]
            V2 = V2_arr[ind-1]
            
        V_est = sp500_m[sp500_m[date_col] == date]['V_est'].values[0]
        sum_sq = 0
        f = lambda V : MSE(g2, X, V[0], V[1], r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                          eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j, Nt, V_est)['avg_err']
        res = minimize(f, [V1, V2], method='L-BFGS-B', bounds= [(0, 1), (0, 1)], tol=1e-4)
        mse += res.fun
        V1_arr[ind] = res.x[0]
        V2_arr[ind] = res.x[1]
        
        print(ind)
        ind += 1
        
    return {'MSE' : mse, 'V1_arr': V1_arr, 'V2_arr': V2_arr}

In [423]:
##############################################################
## Parameters from paper
##############################################################
r = 0.005
div_rate = 0.03
kappa1 = 2
vbar1 = 0.0173
sigma1 = 0.2678
rho1 = -1

kappa2 = 11.5
vbar2 = 0.001
sigma2 = 0.3708
rho2 = -0.72

eta0 = 0.0033
eta1 = 20.27
eta2 = 5.074
mu_x = -0.1241
sigma_x = 0.1141
mu_y = 0.0850
rho_j = -0.3260
    
err_long_dated = get_volatility_states(est_dat, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                 eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j)
print('The RMSE is', np.sqrt(err_long_dated['MSE'] / estimation_dates.shape[0]))


In [ ]:
######################################################################
## Testing on short dated options
######################################################################
err_short_dated = get_volatility_states(short_dated, r, div_rate, kappa1, vbar1, sigma1, rho1, kappa2, vbar2, sigma2, rho2, \
                 eta0, eta1, eta2, mu_x, sigma_x, mu_y, rho_j)
print('The RMSE is', np.sqrt(err_short_dated['MSE'] / short_dated.shape[0]))
V1_arr = err_short_dated['V1_arr']
V2_arr = err_short_dated['V2_arr']

C:\Users\Y\AppData\Local\Programs\Python\Python35\lib\site-packages\ipykernel_launcher.py:112: RuntimeWarning: divide by zero encountered in true_divide


0
1
2


C:\Users\Y\AppData\Local\Programs\Python\Python35\lib\site-packages\ipykernel_launcher.py:112: RuntimeWarning: overflow encountered in true_divide


3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


In [432]:
short_dated[[date_col]].drop_duplicates().shape

(843, 1)

C:\Users\Y\AppData\Local\Programs\Python\Python35\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


89994.27307723269